In [ ]:
import numpy as np 
import pandas as pd
import os


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import librosa
import torch
import librosa.display
import warnings
warnings.filterwarnings("ignore")
# to play the audio files
from IPython.display import Audio
plt.style.use('seaborn-white')

In [ ]:
! pip install transformers

In [ ]:
pip install speechbrain

In [ ]:
crema = "/kaggle/input/cremad/AudioWAV/"
crema_directory_list = os.listdir(crema)
file_name = []
file_emotion = []
file_path = []
file_id = []

for file in crema_directory_list:
    # storing file paths
    file_name.append(file.split('.')[0])
    file_path.append(crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('Sadness')
    elif part[2] == 'ANG':
        file_emotion.append('Anger')
    elif part[2] == 'FEA':
        file_emotion.append('Fear')
    elif part[2] == 'HAP':
        file_emotion.append('Happiness')
    elif part[2] == 'NEU':
        file_emotion.append('Neutral')
    elif part[2] == 'DIS':
        file_emotion.append('Disgust')
    else:
        file_emotion.append('Unknown')
    
    
    file_id.append(int(part[0]))
   

filename_df = pd.DataFrame(file_name, columns=['Name'])
path_df = pd.DataFrame(file_path, columns=['Path'])
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
speaker_id_df = pd.DataFrame(file_id, columns=['ActorID'])

Crema_df = pd.concat([filename_df,path_df,emotion_df, speaker_id_df], axis=1)
Crema_df['source'] = 'cremad'
cdataset = Crema_df
cemotions = ['Sadness', 'Happiness', 'Anger', 'Fear','Disgust', 'Neutral']
cdataset = cdataset[cdataset['Emotions'].isin(cemotions)].reset_index(drop = True)
cdataset['Emotions'].value_counts()

In [ ]:
cdataset

In [ ]:
'''
result = (
  flights_df
  .merge(airports_df, left_on='origin_airport', right_on='iata_code')
)
'''
demographis_df = pd.read_csv('/kaggle/input/cremad-demographics/VideoDemographics.csv')

cdata=cdataset.merge(demographis_df, left_on='ActorID', right_on='ActorID')

In [ ]:
cdata.head


In [ ]:
cdata.drop(labels={'Race','Ethnicity'}, axis=1, inplace=True)

In [ ]:
cdata

In [ ]:
from sklearn import preprocessing
lec = preprocessing.LabelEncoder()
cdataset['Emotion-labels'] = lec.fit_transform(cdataset['Emotions'])
lec_name_mapping = dict(zip(lec.classes_, lec.transform(lec.classes_)))
print(lec_name_mapping)

In [ ]:
CREMA_emotion_summary =cdata.pivot_table(index='Emotions',  aggfunc=len,  values='ActorID')

CREMA_emotion_summary

In [ ]:
CREMA_gender_summary =cdata.pivot_table(index='Sex', aggfunc=len,  values='ActorID' )

CREMA_gender_summary

In [ ]:
CREMA_age_summary =cdata.pivot_table(index='Age',  aggfunc=len,  values='ActorID')

CREMA_age_summary

In [ ]:
CREMA_actor_summary =cdataset.pivot_table(index='ActorID',  aggfunc=len, values = 'source')

CREMA_actor_summary

In [ ]:
import matplotlib.pyplot as plt

cdata['Sex'].value_counts().plot(kind="bar", color=['blue', 'pink'])

# Adding labels to the plot
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Distribution of Genders')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns  # Import Seaborn

# Set Seaborn style
sns.set(style='whitegrid')

# Define a color palette
colors = sns.color_palette('pastel')

plt.figure(figsize=(8, 6))
sns.histplot(data=cdata, x='Age', palette=colors)

# Adding labels to the plot
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Distribution of Ages')  # You might want to change the title

plt.show()



# X vector

In [ ]:
import torchaudio
import torch

# Load the pre-trained x-vector model
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb", run_opts={"device":"cuda"} )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier.to(device)

def extract_xvector(path):
    signal, fs =torchaudio.load(path)
    embeddings = classifier.encode_batch(signal)
    return np.array(embeddings.mean(axis = 0).cpu().squeeze())


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
xv_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_xvector(cdata['Path'][i])
    xv_embeddings.append(features)

xv_embeddings = np.array(xv_embeddings)
print(xv_embeddings.shape)
xv_embedding = np.expand_dims(xv_embeddings, -1)
print(xv_embedding.shape)

# Conformer

In [ ]:
# #Trillson Reference: https://tfhub.dev/google/nonsemantic-speech-benchmark/trillsson4/1
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
m = hub.KerasLayer('https://tfhub.dev/google/trillsson4/1')

import torchaudio
def extract_trillson(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array = np.array(array)
    # NOTE: Audio should be floats in [-1, 1], sampled at 16kHz. Model input is of
    # the shape [batch size, time].
    embeddings = m(array)['embedding']
    # Models internally aggregate over time. For a time-series of embeddings, the
    # user can frame audio however they want.
    embeddings.shape.assert_is_compatible_with([None, 1024])
    embeddings = np.squeeze(np.array(embeddings), axis = 0)
    return embeddings

In [ ]:
from tqdm import tqdm
trillson_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_trillson(cdata['Path'][i])
    trillson_embeddings.append(features)

trillson_embeddings = np.array(trillson_embeddings)
print(trillson_embeddings.shape)

In [ ]:
trillson_embeddings = np.expand_dims(trillson_embeddings, -1)
print(trillson_embeddings.shape)

# MFCC

In [ ]:
#https://www.analyticsvidhya.com/blog/2022/03/implementing-audio-classification-project-using-deep-learning/
from tqdm import tqdm
def extract_mfcc(path):
    samplerate = 16000
    audio, sample_rate = librosa.load(path, sr = samplerate)
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #in order to find out scaled feature we do mean of transpose of value
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

mfcc_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_mfcc(cdata['Path'][i])
    mfcc_embeddings.append(features)

mfcc_embeddings = np.array(mfcc_embeddings)
print(mfcc_embeddings.shape)

mfcc_embedding = np.expand_dims(mfcc_embeddings, -1)
print(mfcc_embedding.shape)

# XLSR

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModel, Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2FeatureExtractor

processor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-xls-r-1b")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-xls-r-1b")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
import torchaudio

def extract_xlsr(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt").to(device)
    # apply the model to the input array from wav
    with torch.no_grad():
       outputs = model(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).cpu().numpy()
    return last_hidden_states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!nvidia-smi --query-gpu=memory.used --format=csv -l

In [ ]:
xlsr_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_xlsr(cdata['Path'][i])
    xlsr_embeddings.append(features)
    

xlsr_embeddings = np.array(xlsr_embeddings)
print(xlsr_embeddings.shape)
xlsr_embedding = np.expand_dims(xlsr_embeddings, -1)
print(xlsr_embedding.shape)

In [ ]:
np.save('/kaggle/working/CREMAD_xv_embeddings.npy', xv_embedding)
np.save('/kaggle/working/CREMAD_mfcc_embeddings.npy', mfcc_embedding)
np.save('/kaggle/working/CREMAD_trillsson_embeddings.npy', trillson_embeddings)

In [ ]:
np.save('/kaggle/working/xlsr_embeddings.npy', xlsr_embedding)

# HuBERT

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModel, Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2FeatureExtractor

processor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
model = Wav2Vec2Model.from_pretrained("facebook/hubert-base-ls960")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

import torchaudio

def extract_Hubert(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt").to(device)
    # apply the model to the input array from wav
    with torch.no_grad():
       outputs = model(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).cpu().numpy()
    return last_hidden_states
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
hubert_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_Hubert(cdata['Path'][i])
    hubert_embeddings.append(features)
    

hubert_embeddings = np.array(hubert_embeddings)
print(hubert_embeddings.shape)
hubert_embedding = np.expand_dims(hubert_embeddings, -1)
print(hubert_embedding.shape)

In [ ]:
np.save('/kaggle/working/hubert_embeddings.npy', hubert_embedding)

# W2V2 SAT

In [ ]:
# Reference; https://bagustris.wordpress.com/2022/08/23/acoustic-feature-extraction-with-transformers/

from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torchaudio

# load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
wavtwovectwo = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)





# audio file is decoded on the fly

def extract_w2v2(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array  = np.array(array)
    array = np.mean(array, axis = 0)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt").to(device)
    # apply the model to the input array from wav
    with torch.no_grad():
        outputs = wavtwovectwo(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).to("cpu").numpy()
    return last_hidden_states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
w2v2_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_w2v2(cdata['Path'][i])
    w2v2_embeddings.append(features)
    

w2v2_embeddings = np.array(w2v2_embeddings)
print(w2v2_embeddings.shape)
w2v2_embedding = np.expand_dims(w2v2_embeddings, -1)
print(w2v2_embedding.shape)

In [ ]:
np.save('/kaggle/working/w2v2_embeddings.npy', w2v2_embedding)

# Unispeech

In [ ]:
from transformers import AutoProcessor, UniSpeechSatModel, Wav2Vec2FeatureExtractor
import torchaudio
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/unispeech-sat-base")
unispeechsat = UniSpeechSatModel.from_pretrained("microsoft/unispeech-sat-base")
unispeechsat.to(device)
def extract_unispeech(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array = np.array(array)
    array = np.mean(array, axis = 0)
    input = processor(array.squeeze(), sampling_rate=sample_rate, return_tensors="pt")
    input = input.to(device)
    with torch.no_grad():
       outputs = unispeechsat(**input)
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).to("cpu").numpy()
    #embeddings = torch.nn.functional.normalize(features, dim=-1).cpu()
    return last_hidden_states
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
unispeech_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_unispeech(cdata['Path'][i])
    unispeech_embeddings.append(features)
    

unispeech_embeddings = np.array(unispeech_embeddings)
print(unispeech_embeddings.shape)
unispeech_embedding = np.expand_dims(unispeech_embeddings, -1)
print(unispeech_embedding.shape)

In [ ]:
np.save('/kaggle/working/unispeech_embeddings.npy', unispeech_embedding)

# WavLM

In [ ]:
from transformers import AutoProcessor, WavLMModel, Wav2Vec2Processor 
import torchaudio
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")
wavlm = WavLMModel.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")# audio file is decoded on the fly
wavlm.to(device)

def extract_wavlm(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array  = np.array(array)
    array = np.mean(array, axis = 0)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt").to(device)
    # apply the model to the input array from wav
    input = input.to(device)
    with torch.no_grad():
       outputs = wavlm(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).to("cpu").numpy()
    return last_hidden_states
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
wavlm_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_wavlm(cdata['Path'][i])
    wavlm_embeddings.append(features)
    

wavlm_embeddings = np.array(wavlm_embeddings)
print(wavlm_embeddings.shape)
wavlm_embedding = np.expand_dims(wavlm_embeddings, -1)
print(wavlm_embedding.shape)

In [ ]:
np.save('/kaggle/working/wavlm_embeddings.npy', wavlm_embedding)

# MMS

In [ ]:
from transformers import AutoProcessor, AutoModelForPreTraining, Wav2Vec2FeatureExtractor
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/mms-1b")
model = Wav2Vec2Model.from_pretrained("facebook/mms-1b")
model.to(device)

def extract_mms(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array  = np.array(array)
    array = np.mean(array, axis = 0)
    input = processor(array.squeeze(), sampling_rate=sample_rate, return_tensors="pt").to(device)
    input = input.to(device)
    with torch.no_grad():
       outputs = model(**input)
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).to("cpu").numpy()
    #embeddings = torch.nn.functional.normalize(features, dim=-1).cpu()
    return last_hidden_states
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
mms_embeddings = []
for i in tqdm(range(len(cdata))):
    features = extract_mms(cdata['Path'][i])
    mms_embeddings.append(features)
    

mms_embeddings = np.array(mms_embeddings)
print(mms_embeddings.shape)
mms_embedding = np.expand_dims(mms_embeddings, -1)
print(mms_embedding.shape)

In [ ]:
np.save('/kaggle/working/CREMAD_mms_embeddings.npy', mms_embedding)